# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Load data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Define features

In [3]:
features = ['RhythmScore', 'AudioLoudness', 'VocalContent', 'AcousticQuality', 
            'InstrumentalScore', 'LivePerformanceLikelihood', 'MoodScore', 
            'TrackDurationMs', 'Energy']
X = train[features]
y = train['BeatsPerMinute']
X_test = test[features]

# Feature engineering: Log-transform skewed features

In [4]:
X['AudioLoudness'] = np.log1p(-X['AudioLoudness'])
X_test['AudioLoudness'] = np.log1p(-X_test['AudioLoudness'])
X['TrackDurationMs'] = np.log1p(X['TrackDurationMs'])
X_test['TrackDurationMs'] = np.log1p(X_test['TrackDurationMs'])

C:\Users\nishi\AppData\Local\Temp\ipykernel_2856\3028610528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['AudioLoudness'] = np.log1p(-X['AudioLoudness'])
C:\Users\nishi\AppData\Local\Temp\ipykernel_2856\3028610528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['AudioLoudness'] = np.log1p(-X_test['AudioLoudness'])
C:\Users\nishi\AppData\Local\Temp\ipykernel_2856\3028610528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

# Split data (80/20 for holdout validation)

In [5]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
X_test_scaled = scaler.transform(X_test)

# Train Individual Models

# Linear Regression

In [7]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_holdout_pred = lr.predict(X_holdout_scaled)
lr_rmse = np.sqrt(mean_squared_error(y_holdout, lr_holdout_pred))
print(f"Linear RMSE: {lr_rmse:.4f}")

Linear RMSE: 26.4440


# Random Forest (your best params)

In [8]:
rf = RandomForestRegressor(n_estimators=150, max_depth=10, min_samples_split=5, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)
rf_holdout_pred = rf.predict(X_holdout_scaled)
rf_rmse = np.sqrt(mean_squared_error(y_holdout, rf_holdout_pred))
print(f"RF RMSE: {rf_rmse:.4f}")


RF RMSE: 26.4398


# XGBoost with tuned params

In [9]:
xgb_model = xgb.XGBRegressor(n_estimators=120, learning_rate=0.05, max_depth=5, random_state=42)
xgb_model.fit(X_train_scaled, y_train)
xgb_holdout_pred = xgb_model.predict(X_holdout_scaled)
xgb_rmse = np.sqrt(mean_squared_error(y_holdout, xgb_holdout_pred))
print(f"XGBoost RMSE: {xgb_rmse:.4f}")

XGBoost RMSE: 26.4439


# LightGBM with tuned params


In [10]:
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, n_jobs=-1)
lgb_model.fit(X_train_scaled, y_train)
lgb_holdout_pred = lgb_model.predict(X_holdout_scaled)
lgb_rmse = np.sqrt(mean_squared_error(y_holdout, lgb_holdout_pred))
print(f"LightGBM RMSE: {lgb_rmse:.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004665 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 419331, number of used features: 9
[LightGBM] [Info] Start training from score 119.056554
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


# Holdout-Guided Weight Optimization (ultra-fine grid)

In [11]:
weight_combos = [
    (0.08, 0.62, 0.28, 0.02), (0.09, 0.61, 0.28, 0.02), (0.10, 0.60, 0.28, 0.02),
    (0.08, 0.61, 0.29, 0.02), (0.09, 0.60, 0.29, 0.02), (0.10, 0.59, 0.29, 0.02),
    (0.08, 0.60, 0.30, 0.02), (0.09, 0.59, 0.30, 0.02), (0.10, 0.58, 0.30, 0.02)
]  # (LR, RF, XGB, LGBM) summing to 1
best_rmse = float('inf')
best_weights = None
for w_lr, w_rf, w_xgb, w_lgb in weight_combos:
    ensemble_holdout_pred = (w_lr * lr_holdout_pred + w_rf * rf_holdout_pred + 
                            w_xgb * xgb_holdout_pred + w_lgb * lgb_holdout_pred)
    ensemble_rmse = np.sqrt(mean_squared_error(y_holdout, ensemble_holdout_pred))
    print(f"Weights (LR: {w_lr}, RF: {w_rf}, XGB: {w_xgb}, LGBM: {w_lgb}) RMSE: {ensemble_rmse:.4f}")
    if ensemble_rmse < best_rmse:
        best_rmse = ensemble_rmse
        best_weights = (w_lr, w_rf, w_xgb, w_lgb)

print(f"Best Weights: {best_weights}, Best Holdout RMSE: {best_rmse:.4f}")

Weights (LR: 0.08, RF: 0.62, XGB: 0.28, LGBM: 0.02) RMSE: 26.4390
Weights (LR: 0.09, RF: 0.61, XGB: 0.28, LGBM: 0.02) RMSE: 26.4389
Weights (LR: 0.1, RF: 0.6, XGB: 0.28, LGBM: 0.02) RMSE: 26.4389
Weights (LR: 0.08, RF: 0.61, XGB: 0.29, LGBM: 0.02) RMSE: 26.4390
Weights (LR: 0.09, RF: 0.6, XGB: 0.29, LGBM: 0.02) RMSE: 26.4389
Weights (LR: 0.1, RF: 0.59, XGB: 0.29, LGBM: 0.02) RMSE: 26.4389
Weights (LR: 0.08, RF: 0.6, XGB: 0.3, LGBM: 0.02) RMSE: 26.4390
Weights (LR: 0.09, RF: 0.59, XGB: 0.3, LGBM: 0.02) RMSE: 26.4390
Weights (LR: 0.1, RF: 0.58, XGB: 0.3, LGBM: 0.02) RMSE: 26.4389
Best Weights: (0.1, 0.6, 0.28, 0.02), Best Holdout RMSE: 26.4389


# Generate test predictions with best weights

In [12]:
lr_test_pred = lr.predict(X_test_scaled)
rf_test_pred = rf.predict(X_test_scaled)
xgb_test_pred = xgb_model.predict(X_test_scaled)
lgb_test_pred = lgb_model.predict(X_test_scaled)
test_pred = (best_weights[0] * lr_test_pred + best_weights[1] * rf_test_pred + 
             best_weights[2] * xgb_test_pred + best_weights[3] * lgb_test_pred)
submission['BeatsPerMinute'] = test_pred
submission.to_csv('submission_ensemble_refined_lgbm.csv', index=False)
print("Submission file created: submission_ensemble_refined_lgbm.csv")

c:\Users\nishi\Downloads\playground-series-s5e9\playground-series-s5e9\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Submission file created: submission_ensemble_refined_lgbm.csv
